<a href="https://colab.research.google.com/github/soiqualang/colab/blob/master/Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Authorization

In [0]:
# Authorize 2 times
# max to 12 hours or 90 mins of inactivity

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create virtual location
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#@title Default title text
# To kill the current virtual machine
!kill -9 -1

## Important packages

In [0]:
# keras
!pip install -q keras

In [0]:
# OpenCV
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [0]:
# pytorch
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# https://github.com/dmlc/xgboost
# This specific version is a work-around for a build issue in newer versions.
!pip install -q xgboost==0.4a30
import xgboost

# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive

# Extract to the current directory and print name of each file
import libarchive.public

for entry in libarchive.public.file_pour('/tmp/test.zip'):
    print(entry)

In [0]:
# MXNet - library for efficient running, outdated version - waiting for the next that support Keras 2
# https://medium.com/@julsimon/keras-shoot-out-tensorflow-vs-mxnet-51ae2b30a9c0
# change keras.json
!cp -rf /content/drive/app/keras.json /content/.keras/keras.json
# check if successful
!cat /content/.keras/keras.json

!pip install keras-mxnet

In [0]:
# Colab Utils: https://github.com/mixuala/colab_utils. Download tboard.py
# Allow to display tensorboard, save and restore current-training checkpoint in Google Cloud
# Change directory to tboard.py directory
os.chdir('/content/drive/Colab Notebooks')
import tboard

# Set up folder to log file
ROOT = "/tmp" # default setting, shouldn't changed
LOG_DIR = os.path.join(ROOT, 'log/1') # should be '/log' directory

# After execute program
# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

!rm -r /tmp/log # remove log file

## Useful command

In [0]:
!pip show tensorflow
# For a specific version:
!pip install tensorflow==1.2

# Run program
!python3 drive/app/examples/mnist_cnn.py

# Draw data
!git clone https://github.com/wxs/keras-mnist-tutorial.git
!wget https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/Titanic.csv -P drive/app

# Check directory
import os
!ls drive/app
os.chdir("drive")
os.chdir("..")

## Check Device Setting

In [0]:
# Number of CPU cores
!grep -c ^processor /proc/cpuinfo
!grep -E 'processor|core id' /proc/cpuinfo # more details

2
processor	: 0
core id		: 0
processor	: 1
core id		: 0


In [0]:
# Device
import tensorflow as tf
from tensorflow.python.client import device_lib
print('Current Device: ', tf.test.gpu_device_name() + '\n')
device_lib.list_local_devices()

# RAM: !cat /proc/meminfo
# CPU: !cat /proc/cpuinfo

Current Device:  /device:GPU:0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6560166980814844637, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 356122624
 locality {
   bus_id: 1
 }
 incarnation: 3485662006225288770
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
# Check alloted memory
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print('Gen RAM Free: ' + humanize.naturalsize( psutil.virtual_memory().available ), ' I Proc size: ' + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

# Current Usage
import os
import psutil
process = psutil.Process(os.getpid())
print("Real usage: ", process.memory_info().rss/(10**9))

  Running setup.py bdist_wheel for gputil ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 10.2 GB  I Proc size: 145.2 MB
GPU RAM Free: 10632MB | Used: 807MB | Util   7% | Total 11439MB
Real usage:  0.145219584


## Dynamic Programming
Don't need to use tf.Session <br>
Run an iteration whenever we want - easier for debugging

In [0]:
# Dynamic programming, eager mode on tensorflow
# Only Native TF are supported in eager mode, meaning can't do it in Keras or tf.python.keras
# Examples: https://github.com/tensorflow/tensorflow/tree/97a4c226e8a9e7c5c36fc38e2b9f8459c77abd5a/tensorflow/contrib/eager/python/examples
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

# Examples of Eager Execution:
https://github.com/ZhuanZhiCode/TensorFlow-Eager-Execution-Examples